<a href="https://colab.research.google.com/github/neo4j-partners/hands-on-lab-neo4j-and-vertex-ai/blob/main/Lab%206%20-%20Vertex%20AI/vertex_ai_embedding.ipynb" target="_blank">
  <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
</a>

# Install additional Packages
First off, you'll also need to install a few packages.

In [1]:
!pip install --quiet google.cloud.aiplatform

     |████████████████████████████████| 1.8 MB 5.5 MB/s 
     |████████████████████████████████| 107 kB 44.2 MB/s 
     |████████████████████████████████| 114 kB 42.1 MB/s 
     |████████████████████████████████| 46 kB 1.9 MB/s 
     |████████████████████████████████| 206 kB 48.0 MB/s 
     |████████████████████████████████| 76 kB 4.4 MB/s 
     |████████████████████████████████| 4.4 MB 38.9 MB/s 
     |████████████████████████████████| 1.1 MB 42.9 MB/s 
Reason for being yanked: Segfaults
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
pandas-gbq 0.13.3 requires google-cloud-bigquery[bqstorage,pandas]<2.0.0dev,>=1.11.1, but you have google-cloud-bigquery 2.34.2 which is incompatible.
google-cloud-translate 1.5.0 requires google-api-core[grpc]<2.0.0dev,>=1.6.0, but you h

# Restart the kernel
After you install the additional packages, you need to restart the notebook kernel so it can find the packages.  When you run this, you may get a notification that the kernel crashed.  You can disregard that.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'restart': True, 'status': 'ok'}

# Authenticate your Google Cloud account


In [1]:
# Edit these variables!
PROJECT_ID = "neo4jbusinessdev"
STORAGE_BUCKET = "form13labbuckjdslkfjlks"

# You can leave these defaults
REGION = 'us-central1'

In [2]:
import os
os.environ["GCLOUD_PROJECT"] = PROJECT_ID

In [4]:
try:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()
except:
    pass

# Train a Model on GCP
We'll use the engineered features to train an AutoML Tables model, then deploy it to an endpoint

In [5]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

dataset = aiplatform.TabularDataset.create(
    display_name="form13embedding",
    gcs_source=os.path.join("gs://", STORAGE_BUCKET, 'embedding', 'train.csv'),
)
dataset.wait()

print(f'\tDataset: "{dataset.display_name}"')
print(f'\tname: "{dataset.resource_name}"')

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/803648085855/locations/us-central1/datasets/7777026063166603264/operations/329785937087168512
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/803648085855/locations/us-central1/datasets/7777026063166603264
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/803648085855/locations/us-central1/datasets/7777026063166603264')
	Dataset: "form13embedding"
	name: "projects/803648085855/locations/us-central1/datasets/7777026063166603264"


In [6]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='train-form13embedding-automl-1',
    optimization_prediction_type='classification'
)

In [ ]:
model = job.run(
    dataset=dataset,
    target_column='target',
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    model_display_name="form13embedding",
    disable_early_stopping=False,
    budget_milli_node_hours=1000,
)

INFO:google.cloud.aiplatform.training_jobs:No column transformations provided, so now retrieving columns from dataset in order to set default column transformations.
INFO:google.cloud.aiplatform.training_jobs:The column transformation of type 'auto' was set for the following columns: ['filingManager', 'embedding_4', 'embedding_13', 'embedding_7', 'embedding_6', 'reportCalendarOrQuarter', 'embedding_8', 'embedding_3', 'embedding_11', 'cusip', 'embedding_10', 'embedding_2', 'embedding_12', 'embedding_5', 'embedding_1', 'value', 'embedding_9', 'embedding_14', 'embedding_0', 'shares', 'embedding_15'].
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5726529786096910336?project=803648085855
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/803648085855/locations/us-central1/trainingPipelines/5726529786096910336 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.

This job will run for an hour.  That's the minimum time for an AutoML job.  We're going to move on to the next notebook.  You can check on the job later in the [Google Cloud Console](https://console.cloud.google.com/) to see the results.  There's a link to the specific job in the output of the cell above.